In [1]:
import pandas as pd
import plotly.express as px

In [16]:
org_data = pd.read_parquet("escooter_history_2022.parquet")

In [5]:
org_data.head()

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
0,2020-01-04 00:00:09,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0,True
1,2020-01-04 00:00:41,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0,True
2,2020-01-04 00:01:20,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0,True
3,2020-01-04 00:04:12,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0,True
4,2020-01-04 00:15:19,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0,True


In [19]:
org_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755415 entries, 0 to 3755414
Data columns (total 9 columns):
 #   Column               Dtype         
---  ------               -----         
 0   datetime             datetime64[ns]
 1   holiday              float64       
 2   workingday           float64       
 3   weather              object        
 4   temp                 float64       
 5   atemp                float64       
 6   humidity             float64       
 7   windspeed            float64       
 8   registered_customer  boolean       
dtypes: boolean(1), datetime64[ns](1), float64(6), object(1)
memory usage: 236.4+ MB


In [22]:
# holiday (float) -> boolean
# wokringday (float) -> boolean
# registered_customer (boolean) -> float

org_data_clean = org_data.copy()

org_data_clean.holiday = org_data.holiday.astype(bool)
org_data_clean.workingday = org_data.workingday.astype(bool)

In [20]:
org_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755415 entries, 0 to 3755414
Data columns (total 9 columns):
 #   Column               Dtype         
---  ------               -----         
 0   datetime             datetime64[ns]
 1   holiday              bool          
 2   workingday           bool          
 3   weather              object        
 4   temp                 float64       
 5   atemp                float64       
 6   humidity             float64       
 7   windspeed            float64       
 8   registered_customer  boolean       
dtypes: bool(2), boolean(1), datetime64[ns](1), float64(4), object(1)
memory usage: 186.2+ MB


,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
datetime,,,,,,,,
2020-01-04,985,985,985,985,985,985,985,985
2020-01-05,801,801,801,801,801,801,801,801
2020-01-06,1349,1349,1349,1349,1349,1349,1349,1349
2020-01-07,1562,1562,1562,1562,1562,1562,1562,1562
2020-01-08,1600,1600,1600,1600,1600,1600,1600,1600
...,...,...,...,...,...,...,...,...
2021-12-30,4231,4231,4231,4231,4231,4231,4231,4231
2021-12-31,8144,8144,8144,8144,8144,8144,8144,8144
2022-01-01,5932,5932,5932,5932,5932,5932,5932,5932
